In [1]:
import cv2
import numpy as np
import pygame

# Initialize Pygame
pygame.init()

# Screen dimensions
screen_width = 600
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Tic-Tac-Toe")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)
blue = (0, 0, 255)

# Game variables
board = np.zeros((3, 3))
current_player = 1  # Player 1 starts (X)

# Function to draw the board
def draw_board():
    screen.fill(white)
    for row in range(1, 3):
        pygame.draw.line(screen, black, (0, 200 * row), (600, 200 * row), 5)
        pygame.draw.line(screen, black, (200 * row, 0), (200 * row, 600), 5)

    for row in range(3):
        for col in range(3):
            if board[row, col] == 1:
                pygame.draw.line(screen, red, (col * 200 + 50, row * 200 + 50), (col * 200 + 150, row * 200 + 150), 5)
                pygame.draw.line(screen, red, (col * 200 + 150, row * 200 + 50), (col * 200 + 50, row * 200 + 150), 5)
            elif board[row, col] == 2:
                pygame.draw.circle(screen, blue, (col * 200 + 100, row * 200 + 100), 50, 5)

# Function to check for a win
def check_win(player):
    for row in range(3):
        if np.all(board[row, :] == player):
            return True
    for col in range(3):
        if np.all(board[:, col] == player):
            return True
    if np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player):
        return True
    return False

# Function to get the board position from coordinates
def get_board_pos(x, y):
    return y // 200, x // 200

# Initialize OpenCV
cap = cv2.VideoCapture(0)

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Read frame from webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define range for a specific color (e.g., red)
    lower_color = np.array([0, 120, 70])
    upper_color = np.array([10, 255, 255])

    # Threshold the HSV image to get only the selected color
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume the largest contour is the hand
    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(max_contour)
        
        # Draw a rectangle around the detected hand
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Get the center of the hand
        cx, cy = x + w // 2, y + h // 2
        
        # Place a mark on the board based on the hand position
        row, col = get_board_pos(cx, cy)
        if 0 <= row < 3 and 0 <= col < 3 and board[row, col] == 0:
            board[row, col] = current_player
            if check_win(current_player):
                print(f"Player {current_player} wins!")
                board = np.zeros((3, 3))
            current_player = 3 - current_player  # Switch player

    draw_board()
    pygame.display.flip()

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()

pygame 2.6.0 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
